In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/MasterThesis/SplittedImages'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [2]:
!git clone https://github.com/joerivc/MasterThesis.git

Cloning into 'MasterThesis'...
remote: Enumerating objects: 4139, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 4139 (delta 3), reused 9 (delta 3), pack-reused 4127
Receiving objects: 100% (4139/4139), 697.00 MiB | 59.71 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Updating files: 100% (6508/6508), done.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
DATADIRTRAIN = '/content/MasterThesis/SplittedImages/train'
DATADIRTEST = '/content/MasterThesis/SplittedImages/test'
DATADIRVAL = '/content/MasterThesis/SplittedImages/val'
CATEGORIES = ['Sitting','SittingPelvicR','Standing','StandingPelvicR']
IMG_SIZE=299

In [5]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRTRAIN, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

In [6]:
test_data=[]
def create_test_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRTEST, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                test_data.append([new_array,class_num])
            except Exception as e:
                pass

In [7]:
val_data=[]
def create_val_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIRVAL, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                val_data.append([new_array,class_num])
            except Exception as e:
                pass

In [8]:
create_training_data()
X_train=[]
y_train=[]
categories_train = []
label_train = []
for categories_train, label_train in training_data:
    X_train.append(categories_train)
    y_train.append(label_train)


In [9]:
X_train1 = np.array(X_train)/255.0
y_train = np.array(y_train)

In [10]:
create_test_data()
X_test=[]
y_test=[]
for categories_test, label_test in test_data:
    X_test.append(categories_test)
    y_test.append(label_test)

In [11]:
X_test1 = np.array(X_test)/255.0

In [12]:
create_val_data()
X_val=[]
y_val=[]
for categories_val, label_val in val_data:
    X_val.append(categories_val)
    y_val.append(label_val)

In [13]:
X_val1 = np.array(X_val)/255.0
y_val = np.array(y_val)

In [14]:
print(X_train1.shape)

(1440, 299, 299, 3)


In [15]:
print(len(y_train))

1440


In [16]:
X_test = np.array(X_test)/255.0
y_test = np.array(y_test)

In [17]:
# Load pretrained Xception model
xception = tf.keras.applications.Xception(weights='imagenet', include_top=True)
xception.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#train_datagen = ImageDataGenerator()
#train_datagen.fit(X_train)
#xception.fit(train_datagen.flow(X_train, y_train, batch_size=32),
#                    steps_per_epoch=len(x_train) / batch_size, epochs=10, verbose=1)
print(X_train1.shape)
xception.fit(X_train1, y_train,epochs=10,batch_size=32,validation_data=(X_val1, y_val))

91884032/91884032 [==============================] - 1s 0us/step
(1440, 299, 299, 3)
Epoch 1/10
45/45 [==============================] - 242s 5s/step - loss: 0.6935 - accuracy: 0.8007 - val_loss: 5.6066 - val_accuracy: 0.4208
Epoch 2/10
45/45 [==============================] - 227s 5s/step - loss: 0.1380 - accuracy: 0.9479 - val_loss: 1.8760 - val_accuracy: 0.7250
Epoch 3/10
45/45 [==============================] - 229s 5s/step - loss: 0.0868 - accuracy: 0.9667 - val_loss: 2.1945 - val_accuracy: 0.6896
Epoch 4/10
45/45 [==============================] - 228s 5s/step - loss: 0.0301 - accuracy: 0.9889 - val_loss: 0.8992 - val_accuracy: 0.7917
Epoch 5/10
45/45 [==============================] - 229s 5s/step - loss: 0.0196 - accuracy: 0.9924 - val_loss: 0.4882 - val_accuracy: 0.8562
Epoch 6/10
45/45 [==============================] - 228s 5s/step - loss: 0.0355 - accuracy: 0.9889 - val_loss: 0.3776 - val_accuracy: 0.9167
Epoch 7/10
45/45 [==============================] - 230s 5s/step - lo

In [18]:
loss, accuracy = xception.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

15/15 [==============================] - 16s 1s/step - loss: 0.5345 - accuracy: 0.8896
Test Loss: 0.5344730019569397, Test Accuracy: 0.8895833492279053
